In [93]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [94]:
spark

In [95]:
cust_df = spark.read.csv("file:///home/hadoop/Downloads/Telco_Customer_Churn.csv",inferSchema=True,header=True)

In [96]:
cust_df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [97]:
cust_df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [98]:
cust_df.cache()

DataFrame[customerID: string, gender: string, SeniorCitizen: int, Partner: string, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: string, Churn: string]

In [99]:
cust_df.count()

7043

In [100]:
cust_df.select('gender').distinct().show()

+------+
|gender|
+------+
|Female|
|  Male|
+------+



In [101]:
from pyspark.sql.functions import col
gender_distribution = cust_df.groupBy("gender").count()
gender_distribution.show()

+------+-----+
|gender|count|
+------+-----+
|Female| 3488|
|  Male| 3555|
+------+-----+



In [102]:
contract_distribution = cust_df.groupBy("Contract").count()
contract_distribution.show()

+--------------+-----+
|      Contract|count|
+--------------+-----+
|Month-to-month| 3875|
|      One year| 1473|
|      Two year| 1695|
+--------------+-----+



In [103]:
yeschurn = cust_df.select('Churn').where("Churn ='Yes'").count()
totchurn = cust_df.select('Churn').count()
perc = (yeschurn/totchurn) * 100
print(perc)

26.536987079369588


In [104]:
from pyspark.sql.functions import *
cust_df.select([count(when(isnull(col), col)).alias(col) for col in cust_df.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [105]:
cust_df = cust_df.withColumn('TotalCharges',when(col('TotalCharges')==" ",None)\
                             .otherwise(col("TotalCharges")))

In [106]:
from pyspark.sql.functions import *
cust_df.select([count(when(isnull(col), col)).alias(col) for col in cust_df.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [107]:
churn_data1 = cust_df.na.drop()

In [108]:
churn_data1.head(5)

[Row(customerID='7590-VHVEG', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='No', tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges='29.85', Churn='No'),
 Row(customerID='5575-GNVDE', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=34, PhoneService='Yes', MultipleLines='No', InternetService='DSL', OnlineSecurity='Yes', OnlineBackup='No', DeviceProtection='Yes', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='One year', PaperlessBilling='No', PaymentMethod='Mailed check', MonthlyCharges=56.95, TotalCharges='1889.5', Churn='No'),
 Row(customerID='3668-QPYBK', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=2, PhoneService='Yes', MultipleL

In [109]:
from pyspark.sql.types import FloatType
churn_data1 = churn_data1.withColumn('TotalCharges',col('TotalCharges').cast(FloatType()))

In [110]:
churn_data1.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: float (nullable = true)
 |-- Churn: string (nullable = true)



In [111]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml import Pipeline

In [112]:
categorical_cols = ["gender","SeniorCitizen","Partner","Dependents","PhoneService","MultipleLines",\
                    "InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",\
                    "StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","Churn"]
# use stringIndexer to convert chategorical columns to numerical indexes
#stage1 = [StringIndexer(inputCol=col_name,outputCol=col_name+"_index")
#        for col_name in categorical_cols]

stages=[]
for catcols in categorical_cols:
    stringindexer = StringIndexer(inputCol=catcols,outputCol=catcols+"Index")
    onehotencoder = OneHotEncoderEstimator(inputCols =[stringIndexer.getOutputCol()],
                                          outputCols=[catcols+"classVec"])
    stages += [stringindexer,onehotencoder]

In [113]:
numericCols = ["tenure","MonthlyCharges","TotalCharges"]
assemblerInputs = [c + "classVec" for c in categorical_cols]+numericCols
assembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")
stages += [assembler]


In [115]:
label_stringIdx = StringIndexer(inputCol='Churn',outputCol='label')
stages += [label_stringIdx]

In [117]:
pipeline = Pipeline(stages=stages)
churn_df = pipeline.fit(churn_data1).transform(churn_data1)

In [119]:
churn_df.select(['features','label']).show(truncate=False)

+---------------------------------------------------------------------------------------------------+-----+
|features                                                                                           |label|
+---------------------------------------------------------------------------------------------------+-----+
|(20,[17,18,19],[1.0,29.85,29.850000381469727])                                                     |0.0  |
|[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,34.0,56.95,1889.5]            |0.0  |
|[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,53.85,108.1500015258789]  |1.0  |
|[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,45.0,42.3,1840.75]            |0.0  |
|(20,[17,18,19],[2.0,70.7,151.64999389648438])                                                      |1.0  |
|(20,[17,18,19],[8.0,99.65,820.5])                                                                  |1.0  |
|[1.0,1.0,1.0,1.0,1.0,1.0,1.

In [121]:
train,test = churn_df.randomSplit([0.8,0.2],seed = 2)

In [122]:
train.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----------+--------------+------------------+---------------------+------------+---------------+---------------+------------------+-----------------+--------------------+------------------+---------------------+--------------------+-----------------------+-------------------+----------------------+-----------------+--------------------+---------------------+------------------------+----------------+-------------------+----------------+-------------------+--------------------+-----------------------+-------------+----------------+---------------------+------------------------+------------------+---------------------+----------+-------------+--------------------+-----+
|customerID|gender|SeniorCitizen|Partner|Depen

In [124]:
from pyspark.ml.classification import DecisionTreeClassifier
tree = DecisionTreeClassifier(featuresCol='features',labelCol='label')
decision_model = tree.fit(train)

In [125]:
predictions = decision_model.transform(test)

In [126]:
predictions.select(['features','label','prediction']).show()

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(20,[17,18,19],[7...|  0.0|       0.0|
|(20,[17,18,19],[7...|  0.0|       0.0|
|(20,[17,18,19],[5...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|(20,[17,18,19],[4...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|(20,[17,18,19],[3...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  1.0|       0.0|
|(20,[17,18,19],[3...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  1.0|       1.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|(20,[17,18,19],[2...|  0.0|       0.0|
|(20,[17,18,19],[2...|  0.0|       0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|       0.0|
|(20,[17,18,19],[6...|  0.0|       0.0|
|(20,[17,18,19],[1...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 20 rows



In [127]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',
                                 metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
accuracy

0.782051282051282